In [78]:
# Importing packages

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import json
import time
import tqdm
import os, sys
import csv

In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [80]:
## Funktioner 


# Define the log function to gather the log information
def log(response,logfile,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','output_file']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{output_path}' + "\n") #Append the information and jump to new line
        


def open_cvr():
    # Åbner CVR-hjemmesiden i virtuel-browser
    url = 'https://datacvr.virk.dk/data/' #Link
    driver = webdriver.Chrome(ChromeDriverManager().install()) #Open virtual browser
    driver.implicitly_wait(100) #Setting waiting
    driver.get(url)#Go to 

    #2

    # Trykker accepter cookies 
    cookie = driver.find_element(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "cpAcceptBtn", " " ))]') #Here we use a CSS selector
    cookie.click()

    return driver  

In [221]:
## Setup, kommer henter url, trykker cookies af, søger og trykker på virksomhed

#1

# Åbner CVR-hjemmesiden i virtuel-browser
url = 'https://datacvr.virk.dk/data/' #Link
driver = webdriver.Chrome(ChromeDriverManager().install()) #Open virtual browser
driver.implicitly_wait(100) #Setting waiting
driver.get(url)#Go to 

#2

# Trykker accepter cookies 
cookie = driver.find_element(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "cpAcceptBtn", " " ))]') #Here we use a CSS selector
cookie.click()

#3

# Trykker på søgefelt
seek = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div/div/form/div/button[1]')
seek.click()


# Trykker på virksomheder
seek = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[2]/div/div/div/fieldset/ul/li[2]/label') #Here we use a CSS selector
seek.click()


/var/folders/ss/pvm65dps049d1q9czm5j5qhm0000gn/T/ipykernel_15584/3892047240.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) #Open virtual browser


In [201]:
### LAVER LISTE MED ALLE DOBBELT KOMBINATION AF ALFABETET ###

from string import ascii_lowercase
import itertools

def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)

alphabeth = []            

for s in iter_all_strings():
    alphabeth.append(s)
    if s == 'zz':
        break
        
alphabeth = alphabeth[26:]

In [202]:
### LAVER LISE MED URL'ER + ALFABET KOMBINATIONER

def change_url(alphabeth):
    url = f'https://datacvr.virk.dk/soegeresultater?fritekst={alphabeth}&sideIndex=0&enhedstype=virksomhed&virksomhedsform=10&virksomhedsstatus=aktiv&size=10'
    return url

url_list = []
for x in alphabeth:
    new_element = change_url(x)
    url_list.append(new_element)

In [219]:
tid = datetime.datetime.now().strftime("%F %H:%M")
tid

'2022-08-11 16:13'

In [222]:
cvr = []
import datetime 

def maskine(url, bogstav):
    driver.get(url)
    with open("scraper_log.log", "a") as f:
        tid = datetime.datetime.now().strftime("%F %H:%M")
        besked = f"Klokken er {tid}. Starter med at scrape..."
        print(besked, file=f); print(besked)
        for i in range(300):
            print(f"Looper på side {i + 1} for bogstav {bogstav}")
            print(f"Looper på side {i + 1} for bogstav {bogstav}", file=f)
            # Laver soup
            driver.implicitly_wait(5)
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source,'lxml')

            # Henter cvr på 
            firma = soup.find_all('div', class_ = "value")

            cvrs = [int(t.text) for t in firma[0::3]]
            for j in cvrs:
                cvr.append(j)

            current_url = driver.current_url
            try:
                next_ = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[4]/div[2]/button[7]') #Here we use a CSS selector
                next_.click()
                time.sleep(0.2)
                new_url = driver.current_url
                if new_url == current_url:
                    print(f"Færdig med at scrape for bogstav {bogstav} (ikke flere sider)")
                    print(f"Færdig med at scrape for bogstav {bogstav} (ikke flere sider)", file=f)
                    break

            except:
                print(f"Færdig med at scrape for bogstav {bogstav} (fangede en error)")
                print(f"Færdig med at scrape for bogstav {bogstav} (fangede en error)", file=f)
                break

            if i% 5 == 0:
                #Gemmer dataframen i en csv_fil
                np.savetxt(f'cvr_scrape_data-{bogstav}.csv', cvr, fmt='% s')
                continue #Continue to next iteration of the loop
    return driver

In [ ]:
for i, url in enumerate(url_list[:10]):
    driver = maskine(url, alphabeth[i])

Klokken er 2022-08-11 16:14. Starter med at scrape...
Looper på side 1 for bogstav aa
Looper på side 2 for bogstav aa
Looper på side 3 for bogstav aa
Looper på side 4 for bogstav aa
Looper på side 5 for bogstav aa
Looper på side 6 for bogstav aa
Looper på side 7 for bogstav aa
Looper på side 8 for bogstav aa
Looper på side 9 for bogstav aa
Looper på side 10 for bogstav aa
Looper på side 11 for bogstav aa
Looper på side 12 for bogstav aa
Looper på side 13 for bogstav aa
Looper på side 14 for bogstav aa
Looper på side 15 for bogstav aa
Looper på side 16 for bogstav aa
Færdig med at scrape for bogstav aa (ikke flere sider)
Klokken er 2022-08-11 16:14. Starter med at scrape...
Looper på side 1 for bogstav ab
Looper på side 2 for bogstav ab
Looper på side 3 for bogstav ab
Looper på side 4 for bogstav ab
Looper på side 5 for bogstav ab
Looper på side 6 for bogstav ab
Looper på side 7 for bogstav ab
Looper på side 8 for bogstav ab
Looper på side 9 for bogstav ab
Looper på side 10 for bogstav 

In [ ]:
# Del 1

    
for bogstav in url_list:
    # Trykker på søgefelt
    seek = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[1]/div/div/form/div/button[1]')
    seek.click()


    # Trykker på virksomheder
    seek = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[2]/div/div/div/fieldset/ul/li[2]/label') #Here we use a CSS selector
    seek.click()
    
    # Søg efter bosgtav 
    logfile = 'log.csv' 
    
    
    for i in range(300):
        
        # Laver soup
        driver.implicitly_wait(5)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,'lxml')

        # Henter cvr på 
        firma = soup.find_all('div', class_ = "value")

        cvrs = [int(t.text) for t in firma[0::3]]
        for j in cvrs:
            cvr.append(j)

        try:
            next_ = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[4]/div[2]/button[7]') #Here we use a CSS selector
            next_.click()
        except:
            break
        if i% 5 == 0:
            #Gemmer dataframen i en csv_fil
            np.savetxt(f'cvr_scrape_data-{bogstav}.csv', cvr, fmt='% s')
            continue #Continue to next iteration of the loop
   

In [145]:
from pathlib import Path

filer = list(Path.cwd().glob("cvr_scrape_data*"))
print(filer)
dfs = [pd.read_csv(file) for file in filer]
pd.concat(dfs, axis=0)

[PosixPath('/Users/frederikwintherbakholt/Documents/GitHub/GruppeISDS22/Eksamen/cvr_scrape_data.csv')]


,26692938
0,30707176
1,28849842
2,36393254
3,33949928
4,16311677
...,...
2944,41248726
2945,38786652
2946,41853174
2947,41219165


In [109]:
# Del 1 ORGINAL 
cvr = []
logfile = 'log.csv'
for i in range(300):
    # Laver soup
    driver.implicitly_wait(5)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'lxml')


    # Henter cvr på 
    firma = soup.find_all('div', class_ = "value")

    cvrs = [int(t.text) for t in firma[0::3]]
    for j in cvrs:
        cvr.append(j)
        
    try:
        next_ = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/div[4]/div[2]/button[7]') #Here we use a CSS selector
        next_.click()
    except:
        break
    if i% 5 == 0:
        #Gemmer dataframen i en csv_fil
        np.savetxt('cvr_test8.csv', cvr, fmt='% s')
        continue #Continue to next iteration of the loop
   

In [110]:
df = pd.read_csv (r'test.csv')
cvr_3 = np.unique(df)

len(cvr_3)

12211